In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Creating new network to update final_bias value by itself
class BasicNN_lightning(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad = False)
        self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad = False)
        self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad = False)

        self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad = False)
        self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad = False)
        self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad = False)

        self.final_bias = nn.Parameter(torch.tensor(0.0), requires_grad = True)

    def forward(self, input):
        input_to_top_relu = input *self.w00+self.b00
        top_relu_output = F.relu(input_to_top_relu)
        scaled_top_relu_output = top_relu_output*self.w01

        input_to_buttom_relu = input * self.w10+self.b10
        buttom_relu_output = F.relu(input_to_buttom_relu)
        scaled_buttom_relu_output = buttom_relu_output*self.w11

        input_to_final_relu = scaled_top_relu_output+scaled_buttom_relu_output+self.final_bias
        output = F.relu(input_to_final_relu)
        return output
    def configure_optimizers(self):
        return SGD(self.parameters(), lr = self.learning_rate)

    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i)
        loss = (output_i -label_i)**2
        return loss




In [ ]:
inputs = torch.tensor([0., -.5, 1.])
labels = torch.tensor([0., 1., 0.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [ ]:


model = BasicNN_lightning()

trainer = L.Trainer(max_epochs = 50)

lr_find_results = trainer.tuner.lr_find(model, train_dataloaders = dataloader,
                                        min_lr = 0.001, max_lr = 1.0, early_stop_threshold= None)

new_lr = lr_find_results.suggestion()
print(f'For learning rate : lr_find() suggests {new_lr:.4f}')


model.learning_rate = new_lr

trainer.fit(model,train_dataloader = dataloader)

In [ ]:
output_values = model(inputs)
sns.set(style='whitegrid')
sns.lineplot(x= inputs,
             y = output_values.detach(),
             color = green,
             linewidth = 2.5)
plt.ylabel('Effectiveness')
plt.xlabel('values')